# Speculative Decoding

SGLang now provides an EAGLE2-based speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

- Official EAGLE code ([SafeAILab/EAGLE](https://github.com/SafeAILab/EAGLE)): ~200 tokens/s
- Standard SGLang Decoding: ~156 tokens/s
- EAGLE Decoding in SGLang: ~297 tokens/s
- EAGLE Decoding in SGLang (w/ `torch.compile`): ~316 tokens/s

All benchmarks below were run on a single H100.

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-17 12:25:25] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=38329, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=452007956, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_

The following error message 'operation scheduled before its operands' can be ignored.
The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-17 12:25:48 TP0] Init torch distributed begin.
[2025-03-17 12:25:48 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-17 12:25:48 TP0] Load weight begin. avail mem=60.57 GB


[2025-03-17 12:25:49 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-03-17 12:25:52 TP0] Init torch distributed begin.
[2025-03-17 12:25:52 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-17 12:25:52 TP0] Load weight begin. avail mem=37.14 GB


[2025-03-17 12:25:52 TP0] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.41s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.41s/it]

[2025-03-17 12:25:54 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=36.21 GB, mem usage=0.93 GB.
[2025-03-17 12:25:54 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-03-17 12:25:54 TP0] Memory pool end. avail mem=35.90 GB
[2025-03-17 12:25:54 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096


[2025-03-17 12:25:54] INFO:     Started server process [744508]
[2025-03-17 12:25:54] INFO:     Waiting for application startup.
[2025-03-17 12:25:54] INFO:     Application startup complete.
[2025-03-17 12:25:54] INFO:     Uvicorn running on http://127.0.0.1:38329 (Press CTRL+C to quit)
[2025-03-17 12:25:54] INFO:     127.0.0.1:33098 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-17 12:25:55] INFO:     127.0.0.1:33114 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-17 12:25:55 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-17 12:26:00 TP0] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-03-17 12:26:02] INFO:     127.0.0.1:33126 - "POST /generate HTTP/1.1" 200 OK
[2025-03-17 12:26:02] The server is fired up and ready to roll!


[2025-03-17 12:26:02] INFO:     127.0.0.1:60188 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [3]:
terminate_process(server_process)

### EAGLE Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--cuda-graph-max-bs`:


In [4]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --cuda-graph-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-17 12:26:20] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=32018, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=436947734, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_r

The following error message 'operation scheduled before its operands' can be ignored.
The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-17 12:26:42 TP0] Init torch distributed begin.


[2025-03-17 12:26:42 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-17 12:26:42 TP0] Load weight begin. avail mem=63.88 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.37s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.37s/it]

[2025-03-17 12:26:48 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.57 GB, mem usage=0.93 GB.
[2025-03-17 12:26:48 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-03-17 12:26:48 TP0] Memory pool end. avail mem=37.25 GB


[2025-03-17 12:26:48 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096


[2025-03-17 12:26:48] INFO:     Started server process [747641]
[2025-03-17 12:26:48] INFO:     Waiting for application startup.
[2025-03-17 12:26:48] INFO:     Application startup complete.
[2025-03-17 12:26:48] INFO:     Uvicorn running on http://127.0.0.1:32018 (Press CTRL+C to quit)


[2025-03-17 12:26:48] INFO:     127.0.0.1:42782 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-17 12:26:49] INFO:     127.0.0.1:42788 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-17 12:26:49 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [5]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-17 12:26:54 TP0] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-03-17 12:26:56] INFO:     127.0.0.1:42800 - "POST /generate HTTP/1.1" 200 OK
[2025-03-17 12:26:56] The server is fired up and ready to roll!


[2025-03-17 12:26:56] INFO:     127.0.0.1:42808 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(server_process)

### EAGLE Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [7]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-17 12:27:12] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=30441, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=806630834, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_

[2025-03-17 12:27:17] Casting torch.bfloat16 to torch.float16.


The following error message 'operation scheduled before its operands' can be ignored.
The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-17 12:27:34 TP0] Casting torch.bfloat16 to torch.float16.


...

[2025-03-17 12:27:54 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192


[2025-03-17 12:27:55] INFO:     Started server process [750795]
[2025-03-17 12:27:55] INFO:     Waiting for application startup.
[2025-03-17 12:27:55] INFO:     Application startup complete.
[2025-03-17 12:27:55] INFO:     Uvicorn running on http://127.0.0.1:30441 (Press CTRL+C to quit)


[2025-03-17 12:27:56] INFO:     127.0.0.1:52882 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-17 12:27:56] INFO:     127.0.0.1:52898 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-17 12:27:56 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-17 12:28:01] INFO:     127.0.0.1:52904 - "POST /generate HTTP/1.1" 200 OK
[2025-03-17 12:28:01] The server is fired up and ready to roll!


In [8]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-17 12:28:01 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-17 12:28:01] INFO:     127.0.0.1:45570 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [9]:
terminate_process(server_process)

[2025-03-17 12:28:01] Child process unexpectedly failed with an exit code 9. pid=751623
[2025-03-17 12:28:01] Child process unexpectedly failed with an exit code 9. pid=751556


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionaly to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [the paper](https://arxiv.org/abs/2406.16858).